In [ ]:
%reset -f
import matplotlib.pyplot as plt
from algorithms import *
from typing import List, Tuple, Callable
from pathlib import Path
import os

In [ ]:
NUMBERS_OF_SELECTION_POINTS = [7, 13, 23, 31, 47, 103]
# DATASETS = ['chelm.txt', 'GlebiaChallengera.csv', '100.csv']
DATASETS = os.listdir('../data')

In [ ]:
def plot_interpolation(
        handler: any,
        data: List[Tuple[float, float]],
        selection_points_strategy: Callable[[float, float, int], List[float]],
        selection_points: int,
        interpolation_method: Callable[[List[Tuple[float, float]], List[float]], List[float]],
        title: str
    ) -> None:
    selected_points = get_selection_points(data, selection_points_strategy, selection_points)
    # x_points = get_linspace_points(data, 512)
    x_points = get_x_values(data)
    y_points = interpolation_method(selected_points, x_points)

    handler.scatter(get_x_values(selected_points), get_y_values(selected_points), color='red', s=10)
    handler.plot(x_points, get_y_values(data), color='green')
    handler.plot(x_points, y_points, color='blue')

    handler.title.set_text(title)
    handler.set_yscale('symlog')
    handler.legend(['Original function', 'Interpolated function', 'Selection points'])
    handler.set_xlabel('Distance, m')
    handler.set_ylabel('Height, m')

In [ ]:
for dataset in DATASETS:
    nodes = read_nodes("../data/" + dataset)
    data_name = dataset.replace(".csv", "").replace(".txt", "")
    Path(f'../plots/{data_name}/').mkdir(parents=True, exist_ok=True)

    # original
    fig, ax = plt.subplots()
    fig.set_size_inches(20, 5)
    ax.plot(get_x_values(nodes), get_y_values(nodes), color='green')
    ax.set_yscale('symlog')
    ax.set_title(f'{dataset} - Original function')
    ax.set_xlabel('Distance, m')
    ax.set_ylabel('Height, m')
    fig.savefig(f'../plots/{data_name}/original.png')

    # linspace nodes
    fig, axs = plt.subplots(3, 2)
    fig.set_size_inches(20, 15)
    fig.suptitle(f'{dataset} - Lagrange interpolation with linearly spaced nodes')
    for i in range(len(NUMBERS_OF_SELECTION_POINTS)):
        plot_interpolation(axs[i//2, i%2], nodes, linspace, NUMBERS_OF_SELECTION_POINTS[i], lagrange_interpolation, f'{NUMBERS_OF_SELECTION_POINTS[i]} selection points')
    fig.savefig(f'../plots/{data_name}/linspace_nodes.png')

    # Chebyshev nodes
    fig, axs = plt.subplots(3, 2)
    fig.set_size_inches(20, 15)
    fig.suptitle(f'{dataset} - Lagrange interpolation with Chebyshev nodes')
    for i in range(len(NUMBERS_OF_SELECTION_POINTS)):
        plot_interpolation(axs[i//2, i%2], nodes, get_chebyshev_nodes, NUMBERS_OF_SELECTION_POINTS[i], lagrange_interpolation, f'{NUMBERS_OF_SELECTION_POINTS[i]} selection points')
    fig.savefig(f'../plots/{data_name}/chebyshev_nodes.png')

    # Cubic spline
    fig, axs = plt.subplots(3, 2)
    fig.set_size_inches(20, 15)
    fig.suptitle(f'{dataset} - Cubic spline interpolation')
    for i in range(len(NUMBERS_OF_SELECTION_POINTS)):
        plot_interpolation(axs[i//2, i%2], nodes, linspace, NUMBERS_OF_SELECTION_POINTS[i], cubic_spline_interpolation, f'{NUMBERS_OF_SELECTION_POINTS[i]} selection points')
    fig.savefig(f'../plots/{data_name}/cubic_spline.png')